In [1]:
import sys
sys.path.append("/home/benjamin/RewardCurriculum/")

import os
os.system("export MKL_SERVICE_FORCE_INTEL=1")

import gymnasium as gym
from collections import defaultdict
import panda_gym
from stable_baselines3.common.vec_env import DummyVecEnv
import json
import numpy as np
import tqdm

import os
from utils.configs import get_config
from utils.env_wrappers import make_vec_env, get_env
from argparse import Namespace
import cv2
from IPython.display import HTML
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

/home/benjamin/miniconda3/envs/rew_curr/lib/python3.10/site-packages/glfw/__init__.py:916: GLFWError: (65544) b'X11: The DISPLAY environment variable is missing'
  warnings.warn(message, GLFWError)
<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead


In [2]:
import matplotlib.font_manager as font_manager

font_dir = ['.']
for font in font_manager.findSystemFonts(font_dir):
    font_manager.fontManager.addfont(font)

In [3]:
def rollout_episodes(env, learner, n_episodes=20, task=None):
    if task is None:
        weights = np.zeros((1, learner.scheduler.reward_dim))
        weights[:, -1] = 1

    rewards = []
    for _ in range(n_episodes):
        terminated = False
        obs = env.reset()
            
        while not terminated:            
            act = learner.predict(obs, weights=weights)[0]
            obs, reward, terminated, info = env.step(act)

        rewards.append(reward)        
    return rewards

In [4]:
os.chdir("/home/benjamin/RewardCurriculum")
results_dir = "/home/benjamin/RewardCurriculum/results/panda_pick_and_place_long"

In [6]:
master_results = defaultdict(list)

env_names = ["PandaMultiRewardPickAndPlaceSphereDense-v3", "PandaMultiRewardPickAndPlaceDense-v3", "PandaMultiRewardPickAndPlaceObstacleDense-v3"]

for subfolder in tqdm.tqdm(os.listdir(results_dir)):
    folder_path = os.path.join(results_dir, subfolder)
    if not os.path.isdir(folder_path):
        continue
    
    args = Namespace()
    args.env_name = json.load(open(os.path.join(folder_path, "config.json"), "r"))["environment"]["env_name"]
    args.seed = json.load(open(os.path.join(folder_path, "config.json"), "r"))["seed"]
    args.config_path = os.path.join(folder_path, "config_original.json")
    args.continue_from = None

    remaining_args = ["--environment.wrapper_kwargs.0.reward_threshold", "-0.05"]
    
    config = get_config(args.config_path, args, remaining_args)
    config["environment"]["wrappers"] += ["SingleTaskRewardWrapper"]
    config["environment"]["wrapper_kwargs"] += [{}]
    
    rewards = []
    for env_name in env_names:
        # if env_name == args.env_name:
        #     continue
        
        make_env_fn = lambda wrappers, wrapper_kwargs, ignore_keyword="ignore" : get_env(env_name, wrappers=wrappers, wrapper_kwargs=wrapper_kwargs, ignore_keyword=ignore_keyword)
        env = make_vec_env(make_env_fn, 
                            n_envs=config["environment"]["n_envs"], 
                            env_kwargs={"wrappers": config["environment"]["wrappers"], "wrapper_kwargs": config["environment"]["wrapper_kwargs"]},
                            monitor_kwargs={"allow_early_resets": True},
                            seed=config["seed"], vec_env_cls=DummyVecEnv)
        
        learner = config["learner_class"].load(os.path.join(folder_path, "evaluations", "best_model.zip"), env=env)
        
        cl_type = str(config["learner_kwargs"]["scheduler_class"]).split(" ")[-1][2:-2]
        rewards = rollout_episodes(env, learner, n_episodes=50)
        master_results[(env_name, cl_type)] += rewards
        

  0%|          | 0/71 [00:00<?, ?it/s]pybullet build time: Sep 15 2023 11:13:12


argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886


/home/benjamin/miniconda3/envs/rew_curr/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:164: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/home/benjamin/miniconda3/envs/rew_curr/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:188: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
/home/benjamin/miniconda3/envs/rew_curr/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:164: UserWarning: WARN: The obs returned by the `step()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/home/benjamin/miniconda3/envs/rew_curr/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:188: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.

argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886


  1%|▏         | 1/71 [00:39<46:36, 39.95s/it]/home/benjamin/miniconda3/envs/rew_curr/lib/python3.10/site-packages/torchrl/__init__.py:32: UserWarning: failed to set start method to spawn, and current start method for mp is fork.
  warn(


...

 97%|█████████▋| 69/71 [37:26<01:14, 37.25s/it]

argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886


 99%|█████████▊| 70/71 [37:58<00:35, 35.64s/it]

argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886


100%|██████████| 71/71 [38:41<00:00, 32.69s/it]


In [7]:
import pickle
pickle.dump(master_results, open(os.path.join(results_dir, "generalization_conditions.pkl"), "wb"))

In [8]:
import pickle
master_results = pickle.load(open(os.path.join(results_dir, "generalization_conditions.pkl"), "rb"))

In [9]:
algorithms = ["SetterSolver", "CurrOT", "ALPGMM"]
env_names = ["PandaMultiRewardPickAndPlaceSphereDense-v3", "PandaMultiRewardPickAndPlaceDense-v3", "PandaMultiRewardPickAndPlaceObstacleDense-v3"]

for env_name in env_names:
    print(env_name)
    for cl_type in algorithms:
        if (env_name, cl_type) not in master_results:
            continue
        
        results = master_results[(env_name, cl_type)]
        results = np.concatenate(results)
        print(results.mean())

PandaMultiRewardPickAndPlaceSphereDense-v3
0.67
0.648
0.676
PandaMultiRewardPickAndPlaceDense-v3
0.976
0.982
0.978
PandaMultiRewardPickAndPlaceObstacleDense-v3
0.548
0.506
0.558
